In [1]:
!wget https://download1478.mediafire.com/zm38laeiz6cg23mwyK2LA4OfF5eyTO4cZ4yQxB7q0aexvM7p2tsl8glaHipVwV1cmM5U0YBXdUnKCeIrNu-rJQzu2YOVZiZG-_OR81P-bG7F8oN2vkvcaqtaVxVyCVPoZhpLDmzLuqixZqBSToNSwJ43Dp2_Yvyro3fwaCV02esa/o8cu4u222zgagwf/ASL_Alph_Redim.zip

--2024-10-12 12:22:28--  https://download1478.mediafire.com/zm38laeiz6cg23mwyK2LA4OfF5eyTO4cZ4yQxB7q0aexvM7p2tsl8glaHipVwV1cmM5U0YBXdUnKCeIrNu-rJQzu2YOVZiZG-_OR81P-bG7F8oN2vkvcaqtaVxVyCVPoZhpLDmzLuqixZqBSToNSwJ43Dp2_Yvyro3fwaCV02esa/o8cu4u222zgagwf/ASL_Alph_Redim.zip
Resolving download1478.mediafire.com (download1478.mediafire.com)... 205.196.123.166
Connecting to download1478.mediafire.com (download1478.mediafire.com)|205.196.123.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1216399571 (1.1G) [application/zip]
Saving to: ‘ASL_Alph_Redim.zip’

ASL_Alph_Redim.zip  100%[===================>]   1.13G  22.8MB/s    in 39s     

2024-10-12 12:23:08 (29.7 MB/s) - ‘ASL_Alph_Redim.zip’ saved [1216399571/1216399571]



In [2]:
!unzip /content/ASL_Alph_Redim.zip

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: ASL_Alph_Redim/Z/Z (3168).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3169).jpg  
  inflating: ASL_Alph_Redim/Z/Z (317).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3170).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3171).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3172).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3173).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3174).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3175).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3176).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3177).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3178).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3179).jpg  
  inflating: ASL_Alph_Redim/Z/Z (318).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3180).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3181).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3182).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3183).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3184).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3185).jpg  
  inflating: ASL_Alph_Redim/Z/Z (3186).jp

In [5]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight

In [4]:
# Ruta al directorio del dataset
data_dir = '/content/ASL_Alph_Redim'

# Parámetros
img_height, img_width = 200, 200
batch_size = 256

# Data augmentation y preprocesamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=(0.1, 0.9),
    fill_mode='nearest',
    validation_split=0.2  # Dividir 75% para entrenamiento y 25% para validación
)

# Crear generadores
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',   # Usar el 80% de los datos para entrenamiento
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',  # Usar el 20% de los datos para validación
    shuffle=False
)

print(f"Cantidad en set de entrenamiento: {train_generator.samples}")
print(f"Cantidad en set de validación: {validation_generator.samples}")

Found 178472 images belonging to 29 classes.
Found 44602 images belonging to 29 classes.
Cantidad en set de entrenamiento: 178472
Cantidad en set de validación: 44602


Guardamos el desbalance de clases

In [6]:
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

Creamos el modelo base

In [8]:
def _conv_model():
  model = models.Sequential()
  # Primera capa convolucional
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
  model.add(layers.MaxPooling2D())

  # Segunda capa convolucional
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D())

  # Aplanamiento y capas densas
  model.add(layers.Flatten())
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dropout(0.5))  # Previene sobreajuste
  model.add(layers.Dense(train_generator.num_classes, activation='softmax'))

  return model

In [10]:
model = _conv_model()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    class_weight=class_weights
)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


698/698 ━━━━━━━━━━━━━━━━━━━━ 2439s 3s/step - accuracy: 0.0586 - loss: 3.4250 - val_accuracy: 0.1400 - val_loss: 2.9391
Epoch 2/15
698/698 ━━━━━━━━━━━━━━━━━━━━ 2450s 3s/step - accuracy: 0.1573 - loss: 2.8048 - val_accuracy: 0.2356 - val_loss: 2.7058
Epoch 3/15
 82/698 ━━━━━━━━━━━━━━━━━━━━ 28:36 3s/step - accuracy: 0.2109 - loss: 2.5632